This notebook assumes that the downloaded data is unzipped inside a folder called `aerial-cactus-identification`

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
from shutil import copyfile
import keras
ORIGIN_PATH=os.getcwd()
TRAIN_ORIGIN_PATH = os.path.abspath(os.path.join("aerial-cactus-identification","train"))
TEST_ORIGIN_PATH = os.path.abspath(os.path.join("aerial-cactus-identification","test"))
META_PATH = os.path.abspath(os.path.join("aerial-cactus-identification","train.csv"))
GENERATOR_PATH = os.path.abspath("TrainData")

# Data fetching & restructuring
**It's necessary to run the next 3 cells initially**

In [6]:
## metaData = pd.read_csv(META_PATH)
try :
    os.mkdir("TrainData")
    os.chdir("TrainData")
    os.mkdir("has_cactus_1")
    os.mkdir("has_cactus_0")
    os.chdir(ORIGIN_PATH)
    os.mkdir("TestData")
    os.chdir("TestData")
    os.mkdir("has_cactus_1")
    os.mkdir("has_cactus_0")
    os.chdir(ORIGIN_PATH)
except :
    pass

In [7]:
destination_1_path_train = os.path.join(ORIGIN_PATH,"TrainData","has_cactus_1")
destination_0_path_train = os.path.join(ORIGIN_PATH,"TrainData","has_cactus_0")
destination_1_path_test = os.path.join(ORIGIN_PATH,"TestData","has_cactus_1")
destination_0_path_test = os.path.join(ORIGIN_PATH,"TestData","has_cactus_0")

***The following code is responsible from separatin the pictures that has cactus and the ones which do not in two different folders. The result is a folder called 'Data' that includes 2 sub folders named as 'has_cactus_1', 'has_cactus_0'***

In [32]:
import random
test_precentage = 0.5
total = len(metaData.values)
progress=0.0
for row in metaData.values:
    progress+=1
    file = row[0]
    label = row[1]
    file_path = os.path.join(TRAIN_ORIGIN_PATH,file)
    if random.random() > (1-test_precentage):
        destination_1_path = destination_1_path_test
        destination_0_path = destination_0_path_test
    else :
        destination_1_path = destination_1_path_train
        destination_0_path = destination_0_path_train
        
    if label == 1:
        dest_path = os.path.join(destination_1_path,file)
        copyfile(file_path,dest_path)
    else :
        dest_path = os.path.join(destination_0_path,file)
        copyfile(file_path,dest_path)
    print('%.3f'%(100*progress/total))

0.006
0.011
0.017
0.023
0.029
0.034
0.040
0.046
0.051
0.057
0.063
0.069
0.074
0.080
0.086
0.091
0.097
0.103
0.109
0.114
0.120
0.126
0.131
0.137
0.143
0.149
0.154
0.160
0.166
0.171
0.177
0.183
0.189
0.194
0.200
0.206
0.211
0.217
0.223
0.229
0.234
0.240
0.246
0.251
0.257
0.263
0.269
0.274
0.280
0.286
0.291
0.297
0.303
0.309
0.314
0.320
0.326
0.331
0.337
0.343
0.349
0.354
0.360
0.366
0.371
0.377
0.383
0.389
0.394
0.400
0.406
0.411
0.417
0.423
0.429
0.434
0.440
0.446
0.451
0.457
0.463
0.469
0.474
0.480
0.486
0.491
0.497
0.503
0.509
0.514
0.520
0.526
0.531
0.537
0.543
0.549
0.554
0.560
0.566
0.571
0.577
0.583
0.589
0.594
0.600
0.606
0.611
0.617
0.623
0.629
0.634
0.640
0.646
0.651
0.657
0.663
0.669
0.674
0.680
0.686
0.691
0.697
0.703
0.709
0.714
0.720
0.726
0.731
0.737
0.743
0.749
0.754
0.760
0.766
0.771
0.777
0.783
0.789
0.794
0.800
0.806
0.811
0.817
0.823
0.829
0.834
0.840
0.846
0.851
0.857
0.863
0.869
0.874
0.880
0.886
0.891
0.897
0.903
0.909
0.914
0.920
0.926
0.931
0.937
0.943
0.949
0.95

7.840
7.846
7.851
7.857
7.863
7.869
7.874
7.880
7.886
7.891
7.897
7.903
7.909
7.914
7.920
7.926
7.931
7.937
7.943
7.949
7.954
7.960
7.966
7.971
7.977
7.983
7.989
7.994
8.000
8.006
8.011
8.017
8.023
8.029
8.034
8.040
8.046
8.051
8.057
8.063
8.069
8.074
8.080
8.086
8.091
8.097
8.103
8.109
8.114
8.120
8.126
8.131
8.137
8.143
8.149
8.154
8.160
8.166
8.171
8.177
8.183
8.189
8.194
8.200
8.206
8.211
8.217
8.223
8.229
8.234
8.240
8.246
8.251
8.257
8.263
8.269
8.274
8.280
8.286
8.291
8.297
8.303
8.309
8.314
8.320
8.326
8.331
8.337
8.343
8.349
8.354
8.360
8.366
8.371
8.377
8.383
8.389
8.394
8.400
8.406
8.411
8.417
8.423
8.429
8.434
8.440
8.446
8.451
8.457
8.463
8.469
8.474
8.480
8.486
8.491
8.497
8.503
8.509
8.514
8.520
8.526
8.531
8.537
8.543
8.549
8.554
8.560
8.566
8.571
8.577
8.583
8.589
8.594
8.600
8.606
8.611
8.617
8.623
8.629
8.634
8.640
8.646
8.651
8.657
8.663
8.669
8.674
8.680
8.686
8.691
8.697
8.703
8.709
8.714
8.720
8.726
8.731
8.737
8.743
8.749
8.754
8.760
8.766
8.771
8.777
8.783
8.78

15.017
15.023
15.029
15.034
15.040
15.046
15.051
15.057
15.063
15.069
15.074
15.080
15.086
15.091
15.097
15.103
15.109
15.114
15.120
15.126
15.131
15.137
15.143
15.149
15.154
15.160
15.166
15.171
15.177
15.183
15.189
15.194
15.200
15.206
15.211
15.217
15.223
15.229
15.234
15.240
15.246
15.251
15.257
15.263
15.269
15.274
15.280
15.286
15.291
15.297
15.303
15.309
15.314
15.320
15.326
15.331
15.337
15.343
15.349
15.354
15.360
15.366
15.371
15.377
15.383
15.389
15.394
15.400
15.406
15.411
15.417
15.423
15.429
15.434
15.440
15.446
15.451
15.457
15.463
15.469
15.474
15.480
15.486
15.491
15.497
15.503
15.509
15.514
15.520
15.526
15.531
15.537
15.543
15.549
15.554
15.560
15.566
15.571
15.577
15.583
15.589
15.594
15.600
15.606
15.611
15.617
15.623
15.629
15.634
15.640
15.646
15.651
15.657
15.663
15.669
15.674
15.680
15.686
15.691
15.697
15.703
15.709
15.714
15.720
15.726
15.731
15.737
15.743
15.749
15.754
15.760
15.766
15.771
15.777
15.783
15.789
15.794
15.800
15.806
15.811
15.817
15.823
15.829

21.931
21.937
21.943
21.949
21.954
21.960
21.966
21.971
21.977
21.983
21.989
21.994
22.000
22.006
22.011
22.017
22.023
22.029
22.034
22.040
22.046
22.051
22.057
22.063
22.069
22.074
22.080
22.086
22.091
22.097
22.103
22.109
22.114
22.120
22.126
22.131
22.137
22.143
22.149
22.154
22.160
22.166
22.171
22.177
22.183
22.189
22.194
22.200
22.206
22.211
22.217
22.223
22.229
22.234
22.240
22.246
22.251
22.257
22.263
22.269
22.274
22.280
22.286
22.291
22.297
22.303
22.309
22.314
22.320
22.326
22.331
22.337
22.343
22.349
22.354
22.360
22.366
22.371
22.377
22.383
22.389
22.394
22.400
22.406
22.411
22.417
22.423
22.429
22.434
22.440
22.446
22.451
22.457
22.463
22.469
22.474
22.480
22.486
22.491
22.497
22.503
22.509
22.514
22.520
22.526
22.531
22.537
22.543
22.549
22.554
22.560
22.566
22.571
22.577
22.583
22.589
22.594
22.600
22.606
22.611
22.617
22.623
22.629
22.634
22.640
22.646
22.651
22.657
22.663
22.669
22.674
22.680
22.686
22.691
22.697
22.703
22.709
22.714
22.720
22.726
22.731
22.737
22.743

28.749
28.754
28.760
28.766
28.771
28.777
28.783
28.789
28.794
28.800
28.806
28.811
28.817
28.823
28.829
28.834
28.840
28.846
28.851
28.857
28.863
28.869
28.874
28.880
28.886
28.891
28.897
28.903
28.909
28.914
28.920
28.926
28.931
28.937
28.943
28.949
28.954
28.960
28.966
28.971
28.977
28.983
28.989
28.994
29.000
29.006
29.011
29.017
29.023
29.029
29.034
29.040
29.046
29.051
29.057
29.063
29.069
29.074
29.080
29.086
29.091
29.097
29.103
29.109
29.114
29.120
29.126
29.131
29.137
29.143
29.149
29.154
29.160
29.166
29.171
29.177
29.183
29.189
29.194
29.200
29.206
29.211
29.217
29.223
29.229
29.234
29.240
29.246
29.251
29.257
29.263
29.269
29.274
29.280
29.286
29.291
29.297
29.303
29.309
29.314
29.320
29.326
29.331
29.337
29.343
29.349
29.354
29.360
29.366
29.371
29.377
29.383
29.389
29.394
29.400
29.406
29.411
29.417
29.423
29.429
29.434
29.440
29.446
29.451
29.457
29.463
29.469
29.474
29.480
29.486
29.491
29.497
29.503
29.509
29.514
29.520
29.526
29.531
29.537
29.543
29.549
29.554
29.560

...

69.429
69.434
69.440
69.446
69.451
69.457
69.463
69.469
69.474
69.480
69.486
69.491
69.497
69.503
69.509
69.514
69.520
69.526
69.531
69.537
69.543
69.549
69.554
69.560
69.566
69.571
69.577
69.583
69.589
69.594
69.600
69.606
69.611
69.617
69.623
69.629
69.634
69.640
69.646
69.651
69.657
69.663
69.669
69.674
69.680
69.686
69.691
69.697
69.703
69.709
69.714
69.720
69.726
69.731
69.737
69.743
69.749
69.754
69.760
69.766
69.771
69.777
69.783
69.789
69.794
69.800
69.806
69.811
69.817
69.823
69.829
69.834
69.840
69.846
69.851
69.857
69.863
69.869
69.874
69.880
69.886
69.891
69.897
69.903
69.909
69.914
69.920
69.926
69.931
69.937
69.943
69.949
69.954
69.960
69.966
69.971
69.977
69.983
69.989
69.994
70.000
70.006
70.011
70.017
70.023
70.029
70.034
70.040
70.046
70.051
70.057
70.063
70.069
70.074
70.080
70.086
70.091
70.097
70.103
70.109
70.114
70.120
70.126
70.131
70.137
70.143
70.149
70.154
70.160
70.166
70.171
70.177
70.183
70.189
70.194
70.200
70.206
70.211
70.217
70.223
70.229
70.234
70.240

76.337
76.343
76.349
76.354
76.360
76.366
76.371
76.377
76.383
76.389
76.394
76.400
76.406
76.411
76.417
76.423
76.429
76.434
76.440
76.446
76.451
76.457
76.463
76.469
76.474
76.480
76.486
76.491
76.497
76.503
76.509
76.514
76.520
76.526
76.531
76.537
76.543
76.549
76.554
76.560
76.566
76.571
76.577
76.583
76.589
76.594
76.600
76.606
76.611
76.617
76.623
76.629
76.634
76.640
76.646
76.651
76.657
76.663
76.669
76.674
76.680
76.686
76.691
76.697
76.703
76.709
76.714
76.720
76.726
76.731
76.737
76.743
76.749
76.754
76.760
76.766
76.771
76.777
76.783
76.789
76.794
76.800
76.806
76.811
76.817
76.823
76.829
76.834
76.840
76.846
76.851
76.857
76.863
76.869
76.874
76.880
76.886
76.891
76.897
76.903
76.909
76.914
76.920
76.926
76.931
76.937
76.943
76.949
76.954
76.960
76.966
76.971
76.977
76.983
76.989
76.994
77.000
77.006
77.011
77.017
77.023
77.029
77.034
77.040
77.046
77.051
77.057
77.063
77.069
77.074
77.080
77.086
77.091
77.097
77.103
77.109
77.114
77.120
77.126
77.131
77.137
77.143
77.149

83.280
83.286
83.291
83.297
83.303
83.309
83.314
83.320
83.326
83.331
83.337
83.343
83.349
83.354
83.360
83.366
83.371
83.377
83.383
83.389
83.394
83.400
83.406
83.411
83.417
83.423
83.429
83.434
83.440
83.446
83.451
83.457
83.463
83.469
83.474
83.480
83.486
83.491
83.497
83.503
83.509
83.514
83.520
83.526
83.531
83.537
83.543
83.549
83.554
83.560
83.566
83.571
83.577
83.583
83.589
83.594
83.600
83.606
83.611
83.617
83.623
83.629
83.634
83.640
83.646
83.651
83.657
83.663
83.669
83.674
83.680
83.686
83.691
83.697
83.703
83.709
83.714
83.720
83.726
83.731
83.737
83.743
83.749
83.754
83.760
83.766
83.771
83.777
83.783
83.789
83.794
83.800
83.806
83.811
83.817
83.823
83.829
83.834
83.840
83.846
83.851
83.857
83.863
83.869
83.874
83.880
83.886
83.891
83.897
83.903
83.909
83.914
83.920
83.926
83.931
83.937
83.943
83.949
83.954
83.960
83.966
83.971
83.977
83.983
83.989
83.994
84.000
84.006
84.011
84.017
84.023
84.029
84.034
84.040
84.046
84.051
84.057
84.063
84.069
84.074
84.080
84.086
84.091

90.109
90.114
90.120
90.126
90.131
90.137
90.143
90.149
90.154
90.160
90.166
90.171
90.177
90.183
90.189
90.194
90.200
90.206
90.211
90.217
90.223
90.229
90.234
90.240
90.246
90.251
90.257
90.263
90.269
90.274
90.280
90.286
90.291
90.297
90.303
90.309
90.314
90.320
90.326
90.331
90.337
90.343
90.349
90.354
90.360
90.366
90.371
90.377
90.383
90.389
90.394
90.400
90.406
90.411
90.417
90.423
90.429
90.434
90.440
90.446
90.451
90.457
90.463
90.469
90.474
90.480
90.486
90.491
90.497
90.503
90.509
90.514
90.520
90.526
90.531
90.537
90.543
90.549
90.554
90.560
90.566
90.571
90.577
90.583
90.589
90.594
90.600
90.606
90.611
90.617
90.623
90.629
90.634
90.640
90.646
90.651
90.657
90.663
90.669
90.674
90.680
90.686
90.691
90.697
90.703
90.709
90.714
90.720
90.726
90.731
90.737
90.743
90.749
90.754
90.760
90.766
90.771
90.777
90.783
90.789
90.794
90.800
90.806
90.811
90.817
90.823
90.829
90.834
90.840
90.846
90.851
90.857
90.863
90.869
90.874
90.880
90.886
90.891
90.897
90.903
90.909
90.914
90.920

96.851
96.857
96.863
96.869
96.874
96.880
96.886
96.891
96.897
96.903
96.909
96.914
96.920
96.926
96.931
96.937
96.943
96.949
96.954
96.960
96.966
96.971
96.977
96.983
96.989
96.994
97.000
97.006
97.011
97.017
97.023
97.029
97.034
97.040
97.046
97.051
97.057
97.063
97.069
97.074
97.080
97.086
97.091
97.097
97.103
97.109
97.114
97.120
97.126
97.131
97.137
97.143
97.149
97.154
97.160
97.166
97.171
97.177
97.183
97.189
97.194
97.200
97.206
97.211
97.217
97.223
97.229
97.234
97.240
97.246
97.251
97.257
97.263
97.269
97.274
97.280
97.286
97.291
97.297
97.303
97.309
97.314
97.320
97.326
97.331
97.337
97.343
97.349
97.354
97.360
97.366
97.371
97.377
97.383
97.389
97.394
97.400
97.406
97.411
97.417
97.423
97.429
97.434
97.440
97.446
97.451
97.457
97.463
97.469
97.474
97.480
97.486
97.491
97.497
97.503
97.509
97.514
97.520
97.526
97.531
97.537
97.543
97.549
97.554
97.560
97.566
97.571
97.577
97.583
97.589
97.594
97.600
97.606
97.611
97.617
97.623
97.629
97.634
97.640
97.646
97.651
97.657
97.663

### Constructing ImageDataGenerators

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.3,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (224, 224),
                                                 batch_size =30,
                                                 class_mode = 'binary',
                                                 subset='training')
validation_set = train_datagen.flow_from_directory(GENERATOR_PATH,
                                                 target_size = (224, 224),
                                                 batch_size =30,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 6084 images belonging to 2 classes.
Found 2606 images belonging to 2 classes.


In [11]:
path=os.path.abspath("TestData")
test_set = ImageDataGenerator().flow_from_directory(path,
                                                 target_size = (224, 224),
                                                 batch_size =30,
                                                 class_mode = 'binary',
                                                 )

Found 8810 images belonging to 2 classes.


# Modeling

## Using Vgg16 model from Keras

In [37]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD

In [38]:
import keras
vgg16_model = keras.applications.vgg16.VGG16()

Instructions for updating:
Colocations handled automatically by placer.


In [39]:
from keras.models import Sequential
model = Sequential()
i = 0
for layer in vgg16_model.layers:
    i +=1
    if i < 23 :
        model.add(layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [40]:
for layer in model.layers:
    layer.trainable=False

In [41]:
from keras.layers import Dense
model.add(Dense(1,activation = 'softmax'))

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [44]:
from keras.optimizers import Adam
model.compile(Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [45]:
model.fit_generator(train_set,steps_per_epoch=200,validation_data=validation_set, validation_steps=65,epochs=5,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
200/200 [==============================] - 2608s 13s/step - loss: 4.0441 - acc: 0.7463 - val_loss: 4.1123 - val_acc: 0.7421
Epoch 2/5
200/200 [==============================] - 2628s 13s/step - loss: 4.0387 - acc: 0.7467 - val_loss: 4.0552 - val_acc: 0.7456
Epoch 3/5
200/200 [==============================] - 2591s 13s/step - loss: 4.0567 - acc: 0.7455 - val_loss: 3.9897 - val_acc: 0.7497
Epoch 4/5
200/200 [==============================] - 2560s 13s/step - loss: 4.0906 - acc: 0.7434 - val_loss: 4.0387 - val_acc: 0.7467
Epoch 5/5
200/200 [==============================] - 2548s 13s/step - loss: 4.0241 - acc: 0.7476 - val_loss: 4.1863 - val_acc: 0.7374


In [46]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [54]:
score = model.evaluate_generator(generator=test_set,steps=293)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

acc: 75.53%


# Manually constructed model

In [13]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD

classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [15]:
classifier.fit_generator(train_set,
                                       steps_per_epoch=200,
                                       validation_data=validation_set,
                                       validation_steps=65,
                                       epochs=5,
                                       verbose=1)

Epoch 1/5
200/200 [==============================] - 369s 2s/step - loss: 0.3423 - acc: 0.8475 - val_loss: 0.2795 - val_acc: 0.9338
Epoch 2/5
200/200 [==============================] - 346s 2s/step - loss: 0.2617 - acc: 0.9200 - val_loss: 0.2307 - val_acc: 0.9414
Epoch 3/5
200/200 [==============================] - 341s 2s/step - loss: 0.2069 - acc: 0.9323 - val_loss: 0.2179 - val_acc: 0.9229
Epoch 4/5
200/200 [==============================] - 342s 2s/step - loss: 0.1767 - acc: 0.9401 - val_loss: 0.1560 - val_acc: 0.9579
Epoch 5/5
200/200 [==============================] - 336s 2s/step - loss: 0.1657 - acc: 0.9461 - val_loss: 0.1441 - val_acc: 0.9497


In [16]:
model_json = classifier.to_json()
with open("cactus_manual_model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
classifier.save_weights("cactus_manual_model.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
score = classifier.evaluate_generator(generator=test_set,steps=290, verbose=1)

290/290 [==============================] - 159s 548ms/step


In [21]:
print("%s: %.2f%%" % (classifier.metrics_names[1], score[1]*100))

acc: 94.68%


In [41]:
from keras.models import model_from_json
# load json and create model
json_file = open('cactus_manual_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("cactus_manual_model.h5")
print("Loaded model from disk")

Loaded model from disk


# Testing

In [43]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate_generator(generator=test_set,steps=293,verbose=1)

293/293 [==============================] - 156s 531ms/step


In [44]:
print("%s: %.2f%%" % (classifier.metrics_names[1], score[1]*100))

acc: 94.81%


# Summary
- The Vgg16 model had an accuracy of 75%, another model should've been used, but since the manually constructed model performed well, I didn't try another one.
- The manually constructed model had an accuracy of 94%